In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

from sklearn.metrics import mean_squared_error

## Task 1

In [2]:
# The training data
dfTrainData = pd.read_csv("./../TrainData.csv")
dfTrainData

,TIMESTAMP,POWER,U10,V10,WS10,U100,V100,WS100
0,20120101 1:00,0.273678,0.534894,-3.660243,3.699120,0.765355,-4.486657,4.551468
1,20120101 2:00,0.086796,0.330813,-2.676430,2.696797,0.469875,-3.213752,3.247920
2,20120101 3:00,0.006811,-0.065839,-2.029072,2.030140,-0.020893,-2.418119,2.418209
3,20120101 4:00,0.018646,-0.419549,-1.799090,1.847362,-0.444867,-2.149654,2.195203
4,20120101 5:00,0.034812,-0.754224,-1.661526,1.824698,-0.838274,-1.991829,2.161038
...,...,...,...,...,...,...,...,...
16075,20131031 20:00,0.101254,-1.459917,-1.778960,2.301316,-1.974368,-4.936280,5.316483
16076,20131031 21:00,0.105047,-1.293326,-2.350447,2.682777,-0.781394,-4.903565,4.965433
16077,20131031 22:00,0.145079,-0.308284,-2.441885,2.461269,0.257905,-4.027432,4.035681
16078,20131031 23:00,0.180933,1.148765,-2.789011,3.016330,1.628004,-3.520358,3.878572


In [3]:
# Limit use of wind speed at 10m above ground level
dfTrainData = dfTrainData.drop(columns=['U100', 'V100', 'WS100'])

In [5]:
dfSolution = pd.read_csv("./../Solution.csv")
dfWFInput= pd.read_csv("./../WeatherForecastInput.csv")

In [50]:
# apply machine learning techniques to find the relationship between wind power generation and wind speed
# apparently, each machine learning technique has a different training model. 

In [6]:
""" Linear Regression """
dfLR = dfTrainData

# Separate features and target variable
x = dfLR.iloc[:, 2:]
y = dfLR.iloc[:, 1]

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Prediction
Train_pred = model.predict(X_test)

# Evaluation of the model
print('Root mean squared error (lr RMSE):', np.round(np.sqrt(mean_squared_error(y_test, Train_pred)), 3))

# # Calculate RMSE
# lr_rmse = np.sqrt(mean_squared_error(y_test, Train_pred), 3)
# print('KNN RMSE:', lr_rmse)

# Assuming y_pred and y_test are defined
lr_predictions_df = pd.DataFrame({ 'Predicted': Train_pred, 'Actual': y_test})

# save to ForecastTemplate1-LR.csv
lr_predictions_df.to_csv('./../ForecastTemplate/ForecastTemplate1-LR.csv', index=False)

Root mean squared error (lr RMSE): 0.225


In [53]:
""" KNN Regression """
# Extract features and target variable
X_train = dfTrainData[['WS10']]  
y_train = dfTrainData['POWER']  

# Train kNN model
knn_model = KNeighborsRegressor(n_neighbors=3)
knn_model.fit(X_train, y_train)

wind_speed_nov_2013 = dfWFInput[['WS10']]  
# Predict wind power for November 2013
wind_power_predictions = knn_model.predict(wind_speed_nov_2013)
# print("Predicted wind power:", wind_power_predictions)

# Save predicted results to a CSV file
knn_predictions_df = pd.DataFrame({'wind_power_prediction': wind_power_predictions})
knn_predictions_df.to_csv('./../ForecastTemplate/ForecastTemplate1-kNN.csv', index=False)
print("Result predictions is saved to 'ForecastTemplate1-kNN.csv'")


true_wind_power_nov_2013 = dfSolution['POWER']
print('Mean squared error (MSE):', np.round(mean_squared_error(true_wind_power_nov_2013, wind_power_predictions), 3))

# Calculate RMSE
knn_rmse = np.sqrt(mean_squared_error(true_wind_power_nov_2013, wind_power_predictions))
print('KNN RMSE:', knn_rmse)

Predictions is saved to 'ForecastTemplate1-kNN.csv'
Mean squared error (MSE): 0.064
KNN RMSE: 0.25201515123405976


In [54]:
""" Support Vector Regression """
# Extract features (wind speed) and target (wind power)
X_train = dfTrainData['WS10'].values.reshape(-1, 1)
y_train = dfTrainData['WS10'].values

# Train SVR model
svr_model = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
svr_model.fit(X_train, y_train)

# Predict wind power generation for November 2013
X_forecast = dfWFInput['WS10'].values.reshape(-1, 1)

y_pred = svr_model.predict(X_forecast)

# Save predicted results in ForecastTemplate1-SVR.cs
svr_predicted = pd.DataFrame({'Predicted Wind Power (kW)': y_pred})
svr_predicted.to_csv("./../ForecastTemplate/ForecastTemplate1-SVR.csv", index=False)
print("Result Prediction is saved in ForecastTemplate1-SVR.csv")

# Calculate RMSE
Acctual_wind_power = dfSolution['POWER'].values
print('Mean squared error (MSE):', np.round(mean_squared_error(Acctual_wind_power, y_pred), 3))
svr_rmse = np.sqrt(mean_squared_error(Acctual_wind_power, y_pred))

print("SVR Model RMSE:", svr_rmse)

     Predicted Wind Power (kW)
0                     3.693199
1                     3.734880
2                     3.740150
3                     3.697232
4                     3.801672
..                         ...
715                   7.095314
716                   7.807490
717                   7.332942
718                   6.606358
719                   5.561499

[720 rows x 1 columns]
Result is saved in ForecastTemplate1-SVR.csv
Mean squared error (MSE): 14.396
SVR Model RMSE: 3.794195642891256


In [55]:
# Artificial Neural Networks (ANN)

print("I couldnt install the package that is needed here, so im not sure about this")
# # save to ForecastTemplate1-NN.csv
# #?????????????????????????? 
# import pyrenn as prn

# # Generate some sample data
# X = np.random.rand(100, 2)  
# y = np.sin(X[:, 0] + X[:, 1])  # Target variable

# # Create a neural network object
# net = prn.NeuralNetwork([2, 5, 1])

# # Train the neural network
# net.train(X, y, epochs=1000, verbose=False)

# # Save the neural network object to a CSV file
# prn.saveNN(net, 'ForecastTemplate1-NN.csv')
# Calculate RMSE for ANN
# rmse_ann = np.sqrt(mean_squared_error(dfSolution['TrueWindPower'], ann_predictions_df))

# print("Model saved to ForecastTemplate1-NN.csv")






In [56]:
"" 'wind speed for the nov'""
# Find the wind speed for the whole month of 11.2013 in WeatherForecastInput.csv
dfWFInput = pd.read_csv("./../WeatherForecastInput.csv")
dfWFInput

# Convert the 'TIMESTAMP' column to datetime format
dfWFInput['TIMESTAMP'] = pd.to_datetime(dfWFInput['TIMESTAMP'])

# Filter the DataFrame to include only the data for November 2013
df_november_2013 = dfWFInput[dfWFInput['TIMESTAMP'].dt.month == 11]
print(df_november_2013)

              TIMESTAMP       U10       V10      WS10      U100       V100  \
0   2013-11-01 01:00:00  3.462251 -1.066374  3.622753  4.323671  -1.330859   
1   2013-11-01 02:00:00  3.651341 -0.337717  3.666925  4.532444  -0.451426   
2   2013-11-01 03:00:00  3.657829  0.328049  3.672510  4.527268   0.341374   
3   2013-11-01 04:00:00  3.551698  0.735371  3.627028  4.395563   0.818799   
4   2013-11-01 05:00:00  3.586921  1.050811  3.737673  4.461128   1.251971   
..                  ...       ...       ...       ...       ...        ...   
714 2013-11-30 19:00:00 -2.824868 -5.425891  6.117203 -4.273966 -10.307238   
715 2013-11-30 20:00:00 -2.527419 -6.623859  7.089666 -3.648235 -11.876879   
716 2013-11-30 21:00:00 -2.529020 -7.423845  7.842794 -3.553146 -12.504219   
717 2013-11-30 22:00:00 -3.105103 -6.653005  7.341943 -4.423704 -12.561661   
718 2013-11-30 23:00:00 -2.286417 -6.165074  6.575396 -2.957473  -9.009961   

         WS100  
0     4.523861  
1     4.554870  
2     4.5401

In [67]:
# Evaluate the prediction accuracy
# Compare the predicted wind power and the true wind power measurements (in the file Solution.csv)
# Use the error metric RMSE to evaluate and compare the prediction accuracy among the machine learning approaches
print("RMSE for ANN and lr should be printed here so that we can compare all of them")
# Print RMSE values for comparison
# print("RMSE for Linear Regression:", lr_predictions_df)
print("RMSE for kNN:", knn_rmse)
print("RMSE for SVR:", svr_rmse)
# print("RMSE for ANN:", rmse_ann)

RMSE for ANN and lr should be printed here so that we can compare all of them
RMSE for kNN: 0.25201515123405976
RMSE for SVR: 3.794195642891256
